In [1]:
import pandas as pd

# create mock input data
# Specify the path to your Excel file
file_path = 'Etapa1 - completat 11.2024.xlsx'

# Read all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(file_path, sheet_name=None)


# Iterate through the dictionary of DataFrames
for sheet_name, df in all_sheets.items():
    # Add a new column with the sheet name
    df['SheetName'] = sheet_name
    
    # Assign the DataFrame to a variable named after the sheet name
    globals()[sheet_name] = df
    globals()[sheet_name].set_index('Primary ID', inplace=True)

In [2]:
cols_prof_cedare = ["%5min", "%10min", "%15min", "%30min", "%60min", "%120min", "%180min", "%240min"]
x = globals()['Input1_Etapa1'].drop(columns=cols_prof_cedare)
compozitie_df = globals()['Output_Etapa1']
X = x.merge(compozitie_df, left_index=True, right_index=True)
columns_to_drop = ['SheetName_x', "SheetName_y"]
X = X.drop(columns=[col for col in columns_to_drop if col in X.columns], errors='ignore')
X.dropna(inplace=True)

# Check the number of columns
num_columns = X.shape[1]
print(f"Number of columns: {num_columns}")

# Create a header list
header = ["Primary ID"] + list(X.columns)

X.to_csv("X_mock.csv", index=True)


Number of columns: 1508


In [3]:
X

,Sec ID,API dose,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,Veber #violations,Egan #violations,Muegge #violations,Bioavailability Score,PAINS #alerts,Brenk #alerts,Leadlikeness #violations,Synthetic Accessibility,API %,Plast %
Primary ID,,,,,,,,,,,,,,,,,,,,,
Amlodipine_N1_1,Amlodipine,7.90,0,0.7160,0.512656,112.2327,60.25982,6,6,53,...,0,0,0,0.55,0,1,2,4.39,5,0.0
Amlodipine_N1_2,Amlodipine,9.30,0,0.7160,0.512656,112.2327,60.25982,6,6,53,...,0,0,0,0.55,0,1,2,4.39,5,0.0
Amlodipine_N2_1,Amlodipine,23.40,0,0.7160,0.512656,112.2327,60.25982,6,6,53,...,0,0,0,0.55,0,1,2,4.39,15,0.0
Amlodipine_N2_2,Amlodipine,22.05,0,0.7160,0.512656,112.2327,60.25982,6,6,53,...,0,0,0,0.55,0,1,2,4.39,15,0.0
Amlodipine_N4_1,Amlodipine,8.25,0,0.7160,0.512656,112.2327,60.25982,6,6,53,...,0,0,0,0.55,0,1,2,4.39,5,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diclofenac_N2_2,Diclofenac,25.95,1,1.8296,3.347436,83.8478,39.03872,12,12,30,...,0,0,0,0.85,0,0,1,2.23,15,0.0
Diclofenac_N4_1,Diclofenac,7.55,1,1.8296,3.347436,83.8478,39.03872,12,12,30,...,0,0,0,0.85,0,0,1,2.23,5,7.5
Diclofenac_N4_2,Diclofenac,8.10,1,1.8296,3.347436,83.8478,39.03872,12,12,30,...,0,0,0,0.85,0,0,1,2.23,5,7.5


In [4]:
from sklearn.decomposition import PCA
import joblib

def infer(X):
    # Drop the columns only if they exist in the DataFrame
    columns_to_drop = ['Sec ID']
    X = X.drop(columns=[col for col in columns_to_drop if col in X.columns], errors='ignore')

    X.set_index('Primary ID', inplace=True)

    # Load the column names from the Excel files (assuming they are in the first row)
    padel_cols = pd.read_excel('Padel_cols.xlsx', header=None).iloc[0].dropna().tolist()
    spartan_cols = pd.read_excel('Spartan_cols.xlsx', header=None).iloc[0].dropna().tolist()
    swissadme_cols = pd.read_excel('Swissadme_cols.xlsx', header=None).iloc[0].dropna().tolist()

    # Load the preprocessing parameters
    zero_variance_columns_padel = joblib.load('padel_cols_to_drop.pkl')
    zero_variance_columns_swissadme = joblib.load('swissadme_cols_to_drop.pkl')
    zero_variance_columns_spartan = joblib.load('spartan_cols_to_drop.pkl')

    # Drop zero variance columns
    X_padel = X[padel_cols].drop(columns=zero_variance_columns_padel)
    X_swissadme = X[swissadme_cols].drop(columns=zero_variance_columns_swissadme)
    X_spartan = X[spartan_cols].drop(columns=zero_variance_columns_spartan)

    # load the values for standardization
    mean_padel = joblib.load('padel_mean.pkl')
    std_padel = joblib.load('padel_std.pkl')
    mean_swissadme = joblib.load('swissadme_mean.pkl')
    std_swissadme = joblib.load('swissadme_std.pkl')
    mean_spartan = joblib.load('spartan_mean.pkl')
    std_spartan = joblib.load('spartan_std.pkl')

    # standardize the molecular descriptor columns
    X_padel_standardized = (X_padel - mean_padel) / std_padel
    X_swissadme_standardized = (X_swissadme - mean_swissadme) / std_swissadme
    X_spartan_standardized = (X_spartan - mean_spartan) / std_spartan

    # load PCA axes
    pca_padel = joblib.load('padel_pca_fit.pkl')
    pca_swissadme = joblib.load('swissadme_pca_fit.pkl')
    pca_spartan = joblib.load('spartan_pca_fit.pkl')

    X_pca_padel = pca_padel.fit_transform(X_padel_standardized)
    X_pca_swissadme = pca_swissadme.fit_transform(X_swissadme_standardized)
    X_pca_spartan = pca_spartan.fit_transform(X_spartan_standardized)

    # Convert PCA results to DataFrames with 'Primary ID' as the index
    X_padel_pca_df = pd.DataFrame(X_pca_padel, index=X.index)
    X_spartan_pca_df = pd.DataFrame(X_pca_spartan, index=X.index)
    X_swissadme_pca_df = pd.DataFrame(X_pca_swissadme, index=X.index)

    other_features_cols = ["API dose", "API %", "Plast %"]
    X_other_features = X[other_features_cols].set_index(X.index)

    # Normalize the rest of the columns
    non_pca_features_mean = joblib.load('non_pca_features_mean.pkl')
    non_pca_features_std = joblib.load('non_pca_features_std.pkl')
    X_standardized = (X_other_features - non_pca_features_mean) / non_pca_features_std
    
    # Merge the PCA-transformed data back together with the rest of the features
    X_final = pd.concat([X_padel_pca_df, X_spartan_pca_df, X_swissadme_pca_df, X_standardized], axis=1)
    print(X_final.columns)

    # Make sure all the column names are strings
    X_final.columns = X_final.columns.astype(str)

    # do inference
    RF_multiregressor = joblib.load('multi_output_model.pkl')
    y_pred = RF_multiregressor.predict(X_final.values)

    return y_pred

    

In [5]:
import pandas as pd

X = pd.read_csv('X_mock.csv')
y_hat = infer(X)

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          0,          1,          2,
                3,          4,          5,          0,          1,          2,
                3,          4,          5,          6, 'API dose',    'API %',
        'Plast %'],
      dtype='object')


In [6]:
print(y_hat.shape)

(220, 8)


In [7]:
y_hat_one = infer(X.iloc[range(0,7), :])


ValueError: n_components=9 must be between 0 and min(n_samples, n_features)=7 with svd_solver='full'

In [8]:
print(X.head())

        Primary ID      Sec ID  API dose  nAcid  ALogP    ALogp2       AMR  \
0  Amlodipine_N1_1  Amlodipine      7.90      0  0.716  0.512656  112.2327   
1  Amlodipine_N1_2  Amlodipine      9.30      0  0.716  0.512656  112.2327   
2  Amlodipine_N2_1  Amlodipine     23.40      0  0.716  0.512656  112.2327   
3  Amlodipine_N2_2  Amlodipine     22.05      0  0.716  0.512656  112.2327   
4  Amlodipine_N4_1  Amlodipine      8.25      0  0.716  0.512656  112.2327   

       apol  naAromAtom  nAromBond  ...  Veber #violations  Egan #violations  \
0  60.25982           6          6  ...                  0                 0   
1  60.25982           6          6  ...                  0                 0   
2  60.25982           6          6  ...                  0                 0   
3  60.25982           6          6  ...                  0                 0   
4  60.25982           6          6  ...                  0                 0   

   Muegge #violations  Bioavailability Score  PAIN